In [2]:
import os
import pandas as pd

# read the text file and add the column names
read_file = pd.read_csv(r"./booksummries.txt", sep='	', header=None)
read_file.columns = ['ID', 'm number', 'book name', 'author name', 'date', 'label', 'summary']

# clean data
read_file['label'] = read_file['label'].str.replace(r'/m/\S*\s', '')
read_file['label'] = read_file['label'].str.replace(r'{', '')
read_file['label'] = read_file['label'].str.replace(r'}', '')
read_file['label'] = read_file['label'].str.replace(r'}', '')
read_file['label'] = read_file['label'].str.replace(r'\\u00e0\s+clef', '')

# select columns
new_file = read_file.loc[:, ['book name', 'label', 'summary']]

#delete the columns with no labels
new_file.dropna(axis = 0, how = 'any', inplace = True)
new_file = new_file.iloc[:, [0, 2, 1]]

#output data as csv
new_file.to_csv(r'./booksummries.csv', index=False)

In [ ]:
def cosine_similarity(a, b):
    return a.dot(b)/np.sqrt(a.dot(a) * b.dot(b))

In [ ]:
def calculation(label_list):
    for i in range(len(label_list)):
        j = i+1
        if (j < len(label_list)):
            for j in range(len(label_list)):
                a = nlp(label_list[i]).vector
                b = nlp(label_list[j]).vector
                if cosine_similarity(a, b)>0.6:
                        if(len(label_list[i])>len(label_list[j])):
                            label_list[i] = label_list[j]
                        else:
                            label_list[j] = label_list[i]
    return set(label_list)

In [ ]:
read_file["label"] = read_file["label"].fillna('')

In [ ]:

for index in range(len(read_file['label'])):
    label = read_file['label'][index].replace('"', ''). lower()
    label_list = re.split(', ', label)  
    label_list = calculation(label_list)
    read_file.xs(index)['label']= label_list

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


def words_process(new_file):
    book_summaries = new_file['summary']
    summary_list = [summary for summary in book_summaries]
    summary_num = len(summary_list)
    #summaries = ''.join(summary_list)
    print("the total number of books: {}\n".format(summary_num))
    
    all_docs = []

    for doc in summary_list:
        # Tokenize the string into words
        tokens = word_tokenize(doc)
        # Remove non-alphabetic tokens, such as punctuation
        words = [word.lower() for word in tokens if word.isalpha()]
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if not word in stop_words]
        all_docs.append(words)
        
    return all_docs

In [ ]:
words_process(new_file.iloc[1:3, :])

**Mehtod1: Using the pre-trained model to obtain the vectors of words and averaging all the words in each summary.**

In [ ]:
from gensim.models import KeyedVectors

# Load word2vec model (trained on an enormous Google corpus)
google_vecs = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)
vec_dim = google_vecs.vector_size

def words_embedding(words):
    

    # Filter the list of vectors to include only those that Word2Vec has a vector for
    vector_list = [google_vecs[word] for word in words if word in google_vecs.vocab]

    # Create a list of the words corresponding to these vectors
    words_filtered = [word for word in words if word in google_vecs.vocab]

    #Zip the words together with their vector representations
    word_vec_zip = zip(words_filtered, vector_list)

    # Cast to a dict so we can turn it into a DataFrame
    word_vec_dict = dict(word_vec_zip)
    word_vec = pd.DataFrame.from_dict(word_vec_dict, orient='index')
    
    return word_vec


In [ ]:
words_embedding(all_docs[1])

In [ ]:
import numpy as np
def avg_doc_embedding(all_docs):
    doc_vec = []
    for doc in all_docs:
        if len(doc) != 0:
            word_vec =  word_embedding(doc)
            doc_vec.append(np.mean(np.array(word_vec), axis=1))
    
    summary_vec = np.array(doc_vec)
    return doc_vec


In [ ]:
avg_doc_embedding(all_docs)

**Method2: Using Word2Vec to train all the words and obtain the vectors. Then combine the TF-DIF model to obtain the weighted vector.**

In [ ]:
import multiprocessing
import sys
from gensim.models import Word2Vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise."
from sklearn.feature_extraction.text import TfidfVectorizer
workers = multiprocessing.cpu_count()
print('number of cpu: {}'.format(workers))

word_model = Word2Vec(all_docs,
                      min_count=2,
                      size=300,
                      window=5,
                      workers=workers,
                      iter=100)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

all_text_docs = []
for doc in all_docs:
    #print(len(doc))
    all_text_docs.append(" ". join(doc))
tf_idf_vect = TfidfVectorizer(stop_words='english', max_features=6000)
final_tf_idf = tf_idf_vect.fit_transform(all_text_docs)
tfidf_feat = tf_idf_vect.get_feature_names()

In [ ]:
tfidf_doc_vectors_ft = [] # the tfidf-ft for each summary  is stored in this list
row=0
errors=0
for doc in all_docs: # for each summary 
    doc_vec = np.zeros(300) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the summary 
    for word in doc: # for each word in a summary 
        try:
            word_vec = word_model.wv[word]
            # obtain the tf_idf of a word in a summary 
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            print(tfidf)
            doc_vec += (word_vec * tfidf)
            weight_sum += tfidf
        except:
            errors =+1
            pass
    doc_vec /= weight_sum
   

    tfidf_doc_vectors_ft.append(doc_vec)
    row += 1
print('errors noted: '+str(errors))

**Method3: Using Doc2Vec to train all the summaries and obtain the doc vectors.**

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn import utils


def doc2vec(all_docs, all_labels):
    
#     doc_labels_dict = dict(zip(all_docs, all_labels))
#     doc_labels = pd.DataFrame.from_dict(doc_labels_dict, orient='index')
    
    summary_tags = []
    for (doc, labels) in zip(all_docs, all_labels):
         summary_tags.append(TaggedDocument(doc, labels))
 
    doc_model = Doc2Vec(dm=1, 
                        vector_size=300, 
                        negative=5, 
                        hs=0, 
                        min_count=2, 
                        sample=0, 
                        workers=workers)
    doc_model.build_vocab(summary_tags)
    for epoch in range(30):
        doc_model.train(utils.shuffle(summary_tags), total_examples=books_number, epochs=1)
        doc_model.alpha -= 0.002
        doc_model.min_alpha = doc_model.alpha
        
    
#     sents = summary_tags.values
    targets, doc_vec = zip(*[(doc.tags, doc_model.infer_vector(doc.words, steps=20)) for doc in summary_tags])
    return targets, doc_vec

targets, doc_vec = doc2vec(all_docs, all_labels)